In [22]:
in_features = 32
out_features = 16


In [23]:
from torch.nn import Linear

classical = Linear(in_features, out_features)
classical.weight, classical.bias


(Parameter containing:
 tensor([[ 0.1594, -0.0521,  0.0043, -0.1027, -0.0943, -0.0552,  0.1466, -0.0770,
           0.1757,  0.0166,  0.0513, -0.0884,  0.1690, -0.0112, -0.0011,  0.1035,
          -0.1184,  0.0792, -0.0940,  0.0448, -0.0268,  0.0602,  0.1435, -0.1468,
           0.0125, -0.0509, -0.0590, -0.0944,  0.1577, -0.0261, -0.0684,  0.0261],
         [-0.1470,  0.1627,  0.0857,  0.0517, -0.1272,  0.0873, -0.1467, -0.0955,
          -0.1554, -0.1357,  0.1632,  0.0747, -0.1634, -0.0167, -0.0083,  0.1468,
          -0.1742,  0.0993,  0.0447, -0.0253, -0.0301, -0.0858, -0.1282,  0.0898,
          -0.1534, -0.1267,  0.0504,  0.1595, -0.0668,  0.1766,  0.0859, -0.0315],
         [-0.0017,  0.1036, -0.0277, -0.1187, -0.1127, -0.0109, -0.0541,  0.1402,
          -0.1763, -0.1722, -0.1371, -0.1643,  0.0863,  0.1699,  0.1202, -0.0003,
          -0.0847, -0.0298,  0.0474,  0.1285,  0.1625, -0.0697, -0.0654,  0.1098,
          -0.0037, -0.0591, -0.0195,  0.0172,  0.0998,  0.1283, -0.0945, 

In [24]:
from qcc.ml.quantum import FullyConnectedLayer, ConvolutionFilter

quantum = FullyConnectedLayer(in_features, out_features, U_kernel=ConvolutionFilter)
quantum.fc.weight, quantum.bias


(Parameter containing:
 tensor([4.6373, 4.3498, 5.5419, 2.6085, 5.0958, 4.5741, 2.8379, 1.5887, 5.1858,
         1.8148, 2.5991, 5.2135, 0.2298, 6.1416, 2.0242, 5.9485, 1.8135, 5.6940,
         5.2367, 0.3847, 5.4828, 4.3732, 1.7968, 0.3559, 4.7206, 0.1702, 6.0865,
         3.3839, 6.1701, 3.3526, 4.7094, 0.4899, 1.3754, 1.1998, 5.3803, 3.2014,
         1.0103, 3.4031, 6.2109, 0.9646, 2.1144, 3.7120, 4.1619, 6.1995, 0.9721,
         1.3407, 0.6436, 2.1752, 2.6144, 5.5595, 2.2695, 2.4033, 0.4929, 0.5402,
         4.4009, 1.8097, 4.0728, 1.3805, 6.1856, 5.1745, 4.8691, 2.4932, 4.7154,
         2.4068, 2.6886, 0.3087, 5.2120, 4.1594, 1.4122, 2.3909, 1.1883, 0.2230,
         1.0002, 3.9882, 5.8532, 3.2120, 0.0601, 2.7478, 0.1837, 0.5664, 5.3580,
         5.3317, 3.8163, 3.9495, 2.1877, 5.1333, 1.5083, 2.9834, 5.4565, 1.3985,
         5.0403, 4.0492, 2.0382, 3.3500, 3.2097, 1.2236, 3.6530, 5.0666, 2.1805,
         4.9286, 0.5437, 5.3566, 0.4286, 6.2682, 3.4079, 3.4614, 5.1726, 1.7141,
     

In [25]:
import torch

weight = torch.abs(classical.weight.T / torch.norm(classical.weight, dim=1)).T
bias = classical.bias

weight = torch.nn.Parameter(weight)
bias = torch.nn.Parameter(bias)

classical.weight, classical.bias = weight, bias
quantum.fc.weight, quantum.bias = torch.nn.Parameter(weight.flatten()), bias
quantum.norm = torch.nn.Parameter(torch.ones_like(bias))

assert((classical.weight.flatten() == quantum.fc.weight).all())
assert((classical.bias == quantum.bias).all())


In [26]:
data = torch.nn.init.uniform_(torch.zeros(in_features))
data = data / torch.abs(data)

In [27]:
(classical.weight @ data) + classical.bias

tensor([4.6539, 4.9559, 4.8065, 4.8797, 5.0050, 4.8798, 4.7300, 4.8756, 4.7849,
        5.0613, 5.0462, 4.8979, 4.9099, 4.7303, 4.8195, 4.8902],
       grad_fn=<AddBackward0>)

In [28]:
classical(data), quantum(data.unsqueeze(0))

(tensor([4.6539, 4.9559, 4.8065, 4.8797, 5.0050, 4.8798, 4.7300, 4.8756, 4.7849,
         5.0613, 5.0462, 4.8979, 4.9099, 4.7303, 4.8195, 4.8902],
        grad_fn=<ViewBackward0>),
 tensor([[4.6539, 4.9559, 4.8065, 4.8797, 5.0050, 4.8798, 4.7300, 4.8756, 4.7849,
          5.0613, 5.0462, 4.8979, 4.9099, 4.7303, 4.8195, 4.8902]],
        grad_fn=<ToCopyBackward0>))